In [72]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def generate_pro_session_dataset(target_rows=1550127):
    print(f"🚀 Generando {target_rows:,} registros con SKU obligatorio en Page View...")
    
    event_master = {
        'home_page': 1, 'log_in': 2, 'page_view': 3, 'scroll': 4, 
        'click_product': 5, 'add_to_cart': 6, 'begin_checkout': 7, 
        'end_checkout': 8, 'purchase': 9
    }
    
    funnel_steps = list(event_master.keys())
    tz_map = {'Chile': 'UTC-3', 'México': 'UTC-6', 'España': 'UTC+1', 'Argentina': 'UTC-3', 'Colombia': 'UTC-5'}
    
    all_data = []
    base_date = datetime(2026, 1, 1)
    n_users = 250000 
    row_id = 1 

    for i in range(n_users):
        u_id = 11000000 + i # IDs consistentes
        u_country = np.random.choice(list(tz_map.keys()))
        u_tz = tz_map[u_country]
        u_device = np.random.choice(['Mobile', 'Desktop', 'Tablet'])
        u_source = np.random.choice(['Google', 'Facebook', 'Direct', 'Email'])
        
        for s in range(np.random.randint(1, 5)):
            u_session_id = f"SESS-{u_id}-{s+1}"
            
            # Definimos el producto de la sesión ANTES de los eventos
            u_category = np.random.choice(['Electronics', 'Clothing', 'Home', 'Beauty'])
            u_sku = f"SKU-{np.random.randint(1000, 5000)}"
            u_price = round(np.random.uniform(20.0, 500.0), 2)
            u_cart_id = f"CART-{np.random.randint(100000, 999999)}"
            
            u_time = base_date + timedelta(days=np.random.randint(0, 31), hours=np.random.randint(0, 24))
            n_steps = np.random.choice(range(1, 10), p=[0.2, 0.15, 0.15, 0.15, 0.1, 0.1, 0.07, 0.05, 0.03])
            
            for step_idx in range(n_steps):
                event_name = funnel_steps[step_idx]
                event_type_id = event_master[event_name]
                
                # Ajuste de tiempos según tu instrucción (decisión de compra toma más tiempo)
                if event_name in ['page_view', 'click_product']:
                    wait = np.random.randint(60, 600) # 1 a 10 min
                else:
                    wait = np.random.randint(10, 60)
                
                u_time += timedelta(seconds=wait)
                
                # --- LÓGICA DE CONGRUENCIA DE SKU ---
                # Ahora el SKU y Category aparecen obligatoriamente desde el paso 3 (page_view)
                is_viewing_product = step_idx >= 2 
                is_transactional = step_idx >= 5 
                
                all_data.append({
                    'row_id': row_id,
                    'event_type_id': event_type_id,
                    'user_id': u_id,
                    'session_id': u_session_id,
                    'event_name': event_name,
                    'event_timestamp': u_time,
                    'time_zone': u_tz,
                    'country': u_country,
                    'device': u_device,
                    'traffic_source': u_source,
                    'category': u_category if is_viewing_product else None,
                    'sku': u_sku if is_viewing_product else None,
                    'cart_id': u_cart_id if is_transactional else None,
                    'units': 1 if is_transactional else 0,
                    'amount': u_price if is_transactional else 0.0
                })
                row_id += 1
                if len(all_data) >= target_rows: break
            if len(all_data) >= target_rows: break
        if len(all_data) >= target_rows: break

    df = pd.DataFrame(all_data)
    return df.sample(frac=1).reset_index(drop=True)

df_final = generate_pro_session_dataset(1550127)
df_final.to_csv('data_clase_sql_final.csv', index=False)

🚀 Generando 1,550,127 registros con SKU obligatorio en Page View...


In [92]:
# 1. Encontrar IDs de usuarios que tienen más de una sesión única
usuarios_multi_sesion = df_final.groupby('user_id')['session_id'].nunique()
ids_activos = usuarios_multi_sesion[usuarios_multi_sesion == 1].index.tolist()

if not ids_activos:
    print("No se encontraron usuarios con múltiples sesiones. Revisa la generación.")
else:
    # Elegimos el primer ID de la lista de usuarios activos
    target_id = ids_activos[0]
    
    # Filtramos la tabla original por ese ID y ORDENAMOS por tiempo
    journey_completo = df_final[df_final['user_id'] == target_id].sort_values('event_timestamp')
    
    print(f"✅ Usuario encontrado: {target_id}")
    print(f"📊 Total de sesiones de este usuario: {usuarios_multi_sesion[target_id]}")
    display(journey_completo)

✅ Usuario encontrado: 11000006
📊 Total de sesiones de este usuario: 1


,row_id,event_type_id,user_id,session_id,event_name,event_timestamp,time_zone,country,device,traffic_source,category,sku,cart_id,units,amount
671132,79,1,11000006,SESS-11000006-1,home_page,2026-01-27 11:00:51,UTC+1,España,Desktop,Facebook,None,None,None,0,0.0
1462409,80,2,11000006,SESS-11000006-1,log_in,2026-01-27 11:01:13,UTC+1,España,Desktop,Facebook,None,None,None,0,0.0
1381930,81,3,11000006,SESS-11000006-1,page_view,2026-01-27 11:10:28,UTC+1,España,Desktop,Facebook,Home,SKU-4343,None,0,0.0


In [89]:
df_final[df_final['user_id'] == 11083493].head()

,row_id,event_type_id,user_id,session_id,event_name,event_timestamp,time_zone,country,device,traffic_source,category,sku,cart_id,units,amount
125,796217,9,11083493,SESS-11083493-3,purchase,2026-01-07 12:16:08,UTC-5,Colombia,Tablet,Email,Home,SKU-3563,CART-730515,1,343.75
137493,796216,8,11083493,SESS-11083493-3,end_checkout,2026-01-07 12:15:12,UTC-5,Colombia,Tablet,Email,Home,SKU-3563,CART-730515,1,343.75
152970,796205,2,11083493,SESS-11083493-2,log_in,2026-01-11 22:01:28,UTC-5,Colombia,Tablet,Email,None,None,None,0,0.00
377974,796208,5,11083493,SESS-11083493-2,click_product,2026-01-11 22:12:15,UTC-5,Colombia,Tablet,Email,Beauty,SKU-3809,None,0,0.00
630822,796206,3,11083493,SESS-11083493-2,page_view,2026-01-11 22:08:28,UTC-5,Colombia,Tablet,Email,Beauty,SKU-3809,None,0,0.00


In [12]:
import pandas as pd
import sqlite3
import time
from tqdm.notebook import tqdm
from IPython.core.magic import register_cell_magic
from IPython.display import display, HTML, clear_output

# 1. Configuración de enlaces (URL LFS Directa)
datasets = {
    'ecommerce_events': "https://media.githubusercontent.com/media/hector1994/e_commerce_funnel_data_generator/refs/heads/master/data_clase_sql_final.csv"
}

# 2. Conexión a la base de datos en memoria
# Aumentamos el timeout para procesar el gran volumen de datos
connector = sqlite3.connect(':memory:', check_same_thread=False)

# 3. Proceso de carga con Barra de Progreso
print("⬇️🗂️ Descargando y sincronizando Dataset Masivo (1.5M registros).....🔄⚙️")
summary_data = []

for name, url in tqdm(datasets.items(), desc="Cargando Tablas"):
    # Descarga de datos
    df = pd.read_csv(url)

    # --- CAMBIO IMPORTANTE: DEFINICIÓN DE FECHAS ---
    # Convertimos la columna de timestamp a formato datetime real
    if 'event_timestamp' in df.columns:
        df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])

    # Conversión a SQL (con chunksize para optimizar memoria)
    df.to_sql(name, connector, index=False, if_exists='replace', chunksize=10000)

    # Guardar info para el resumen
    summary_data.append({
        "Table Name": f"{name}",
        "Rows": f"{len(df):,}",
        "Columns": len(df.columns)
    })

# 4. Definición de la "Palabra mágica" %%sql
@register_cell_magic
def sql(line, cell):
    try:
        # Ejecución de la consulta
        resultado = pd.read_sql(cell, connector)
        clear_output(wait=True)
        display(HTML("<b style='color: #4CAF50;'>✅ Query completed successfully:</b>"))
        return display(resultado)
    except Exception as e:
        clear_output(wait=True)
        display(HTML(f"<b style='color: #F44336;'>❌ Query execution failed:</b><br><code style='color: grey;'>{str(e)}</code>"))

# 5. Interfaz final para el alumno
clear_output()
display(HTML("<h2 style='color: #8e62f3'>✨ Database initialized - E-commerce Funnel Data ✨</h2>"))
display(HTML("<p>The 1.5M rows dataset has been loaded. You can now use <b>ecommerce_events</b> in your queries:</p>"))

# Mostramos el resumen
display(pd.DataFrame(summary_data))
print("\n ✨🚀 SYSTEM READY! START ANALYZING THE FUNNEL! 🚀✨")

,Table Name,Rows,Columns
0,ecommerce_events,"1,500,000",13



 ✨🚀 SYSTEM READY! START ANALYZING THE FUNNEL! 🚀✨


In [27]:
%%sql


SELECT *
FROM ecommerce_events LIMIT 10


,user_id,session_id,event_timestamp,event_name,country,device,traffic_source,category,sku,cart_id,units,amount,payment_method
0,USR-134178,SES-4170785779,2023-12-26 04:08:32,home_page,BRA,Desktop,paid_ad,None,None,CRT-7269685,None,None,None
1,USR-134178,SES-4170785779,2023-12-26 04:12:34,page_view,None,Desktop,None,Home,SKU-HO-3601,CRT-7269685,None,None,None
2,USR-134178,SES-4170785779,2023-12-26 04:13:00,scroll,BRA,Desktop,paid_ad,None,None,CRT-7269685,None,None,None
3,USR-531482,SES-1288335115,2023-04-02 10:26:18,home_page,CHL,Mobile,paid_ad,None,None,CRT-14245098,None,None,None
4,USR-681093,SES-672998665,2024-04-11 13:29:56,home_page,MEX,Mobile,social,None,None,CRT-9068504,None,None,None
5,USR-681093,SES-672998665,2024-04-11 13:34:20,page_view,MEX,Mobile,social,Home,SKU-HO-6375,CRT-9068504,None,None,None
6,USR-782939,SES-1469959690,2025-03-13 16:26:25,home_page,BRA,Mobile,email,None,None,CRT-11962909,None,None,None
7,USR-782939,SES-1469959690,2025-03-13 16:30:34,page_view,BRA,Mobile,email,Fashion,SKU-FA-1356,CRT-11962909,None,None,None
8,USR-782939,SES-1469959690,2025-03-13 16:33:23,scroll,BRA,Mobile,email,None,None,CRT-11962909,None,None,None
9,USR-782939,SES-1469959690,2025-03-13 16:36:36,click_product,BRA,Mobile,email,Fashion,SKU-FA-1356,CRT-11962909,None,None,None
